In [5]:
import pandas as pd
from sqlalchemy import create_engine
import mmh3
# 线程池
from concurrent.futures import ThreadPoolExecutor
# 步骤2: 使用SQLAlchemy创建数据库连接
# 请根据您的StarRocks数据库信息替换以下占位符
username = 'root'
password = ''  # 如果您设置了密码，请在这里填写
host = '172.31.98.9'
port = 9030    # StarRocks默认的MySQL协议端口是9030，但这里使用8030，如果不同请修改
database = 'tron_usdt_trans'  # 您的数据库名

# 创建数据库引擎

engine = create_engine(f'mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}')
# df分块
# 步骤3: 将DataFrame写入StarRocks表中
# 请确保表 'weatherdata' 已经存在于您的数据库中，并且其列与DataFrame中的列匹配

dfs = []
adds = ['TXLWDuAP3F7xVjqfZPwGrnvTBYHTGYT1Qu', 'TPGZHxmFwHwNQHRHr5pb7SsdHJB4jVhi1k','TNj3LgXWLMp1oVdfCNKFkPKfUrCCqftcGr','TWrxonF66TALQwJzPiJCqLstdZb3v533DY', 'TLRhiCwfqHbEdLqEDBsUA83JQPJAhyr7yQ','TL7QUBTHo9tmqdzyDXod3ikQuLhH75QFAS']
for add in adds:
    add_hash = mmh3.hash64(add)[0]
    df = pd.read_sql(f"select * from usdt_trading_details where `from_hash`= {add_hash} or `to_hash`={add_hash}", con=engine)
    dfs.append(df)

In [99]:
df = pd.concat(dfs)
df = df.sort_values('local_time', ascending=True)

In [2]:
import pandas as pd
from sqlalchemy import create_engine
import mmh3
# 线程池
from concurrent.futures import ThreadPoolExecutor
# 步骤2: 使用SQLAlchemy创建数据库连接
# 请根据您的StarRocks数据库信息替换以下占位符
username = 'root'
password = ''  # 如果您设置了密码，请在这里填写
host = '172.31.98.9'
port = 9030    # StarRocks默认的MySQL协议端口是9030，但这里使用8030，如果不同请修改
database = 'tron_usdt_trans'  # 您的数据库名

# 创建数据库引擎

engine = create_engine(f'mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}')
# df分块
# 步骤3: 将DataFrame写入StarRocks表中
# 请确保表 'weatherdata' 已经存在于您的数据库中，并且其列与DataFrame中的列匹配

dfs = []
# adds = ['TXLWDuAP3F7xVjqfZPwGrnvTBYHTGYT1Qu', 'TPGZHxmFwHwNQHRHr5pb7SsdHJB4jVhi1k','TNj3LgXWLMp1oVdfCNKFkPKfUrCCqftcGr','TWrxonF66TALQwJzPiJCqLstdZb3v533DY', 'TLRhiCwfqHbEdLqEDBsUA83JQPJAhyr7yQ','TL7QUBTHo9tmqdzyDXod3ikQuLhH75QFAS']
adds = ['TXLWDuAP3F7xVjqfZPwGrnvTBYHTGYT1Qu','TNj3LgXWLMp1oVdfCNKFkPKfUrCCqftcGr']
for add in adds:
    # add_hash = mmh3.hash64(add)[0]
    # df = pd.read_sql(f"select * from usdt_trading_details where `from_hash`= {add_hash} or `to_hash`={add_hash}", con=engine)
    df = pd.read_csv(f'{add}_transactions1.csv')
    dfs.append(df)

df = pd.concat(dfs)
df.shape
df = df.sort_values('block_timestamp', ascending=True).reset_index()

In [22]:
t1 = df[df['from'] == 'TXLWDuAP3F7xVjqfZPwGrnvTBYHTGYT1Qu']['to'].drop_duplicates()
t2 = df[df['from'] == 'TNj3LgXWLMp1oVdfCNKFkPKfUrCCqftcGr']['to'].drop_duplicates()
t3 = set(t1) & set(t2)

In [26]:
t3 = list(t3)
t3.append('TLJEs5a3eLdh7g6NR8NgnxfCVxcBLfyQ17')

In [27]:
import requests

# API请求地址
base_url = "https://www.oklink.com/api/v5/explorer/address/entity-label"

# 你的API-key
api_key = "c1d7c714-1669-41c5-bd2e-f836ed5461b3"

# 公链缩写符号
chain_short_name = "tron"

# 地址列表，假设有50个地址
addresses = t3

# 分组处理，每组最多20个地址
address_groups = [addresses[i:i + 20] for i in range(0, len(addresses), 20)]

# 存储结果
binance_addresses = []

for group in address_groups:
    # 构造请求参数
    params = {
        "chainShortName": chain_short_name,
        "address": ",".join(group)
    }
    # 发送请求
    headers = {"Ok-Access-Key": api_key}
    response = requests.get(base_url, headers=headers, params=params)

    # 解析响应
    if response.status_code == 200:
        data = response.json().get("data", [])
        for item in data:
            label = item.get("label", "")
            address = item.get("address", "")
            binance_addresses.append({"label": label, "address": address})
    else:
        print(f"请求失败，状态码: {response.status_code}")
        print(response.text)

# 输出结果
for addr in binance_addresses:
    print(f"地址: {addr['address']}, 标签: {addr['label']}")

In [30]:
pd.DataFrame(t3)

,0
0,TRzCiXafzK5HK575mCY5r1BURE6raefj6z
1,TQZKhUgPB9d1BKp385aoaqUofMQqoAVnDE
2,TVYaxDNuyrwgQU9xosfRDaB3KEtW2HXJpJ
3,TXWxkGuz7GoocD5vRxPqZgCgrPScwFwzHS
4,TM58PXhaFnM8xm6G7wjwU6ft1HDRqySK4Q
...,...
136,TTNbPjp7fNTCTDozUaFxUsgtTGe8vgZ7w9
137,TQDAYNbc8of7ZzNEFxH7ku6wtYwnAp4uVW
138,TJ9juhVLaJ13AJxVFxXGnBsD456GjXh4sQ
139,TDmU7C25TSfxujhXoTPDySKrBC2Ko3XTWi


In [167]:
import math
from collections import Counter
class Node:
    def __init__(self, address, is_dirty):
        self.address = address
        self.balance = {'dirty': 0.0, 'clear': 0.0}  # 使用字典替换Counter
        self.is_dirty = is_dirty

    def receive_money(self, money):
        if self.address == 'TL7QUBTHo9tmqdzyDXod3ikQuLhH75QFAS':
            print(money, self.balance)
        self.balance['dirty'] += money.get('dirty', 0.0)
        self.balance['clear'] += money.get('clear', 0.0)

    def send_dirty_money(self, money, node=None):
        if self.balance['dirty'] < money:
            send_money = {
                'dirty': self.balance['dirty'],
                'clear': money - self.balance['dirty']
            }
        else:
            send_money = {
                'dirty': money,
                'clear': 0.0
            }

        self.balance['dirty'] -= send_money['dirty']
        self.balance['clear'] -= send_money['clear']
        # 如果小于0，则变为零
        # if self.balance['clear'] < 0 and self.address == 'TL7QUBTHo9tmqdzyDXod3ikQuLhH75QFAS':
        #     print(send_money, self.balance)
            # self.balance['clear'] = 0
        # self.balance['dirty'] = abs(self.balance['dirty'])
        # self.balance['clear'] = abs(self.balance['clear'])
        if node:
            node.receive_money(send_money)

    def send_clear_money(self, money, node=None):
        if self.balance['clear'] < money:
            send_money = {
                'dirty': money - self.balance['clear'],
                'clear': self.balance['clear']
            }
        else:
            send_money = {
                'dirty': 0.0,
                'clear': money
            }
        if self.balance['dirty'] - money + self.balance['clear'] < 0 and self.address == 'TL7QUBTHo9tmqdzyDXod3ikQuLhH75QFAS':
            print(money, send_money, self.balance)
        self.balance['dirty'] -= send_money['dirty']
        self.balance['clear'] -= send_money['clear']
        # 如果小于0，则变为零
        if self.balance['dirty'] < 0 and self.address == 'TL7QUBTHo9tmqdzyDXod3ikQuLhH75QFAS':
            print(send_money, self.balance)
            # send_money['dirty'] = 0
        # self.balance['dirty'] = abs(self.balance['dirty'])
        # self.balance['clear'] = abs(self.balance['clear'])
        if node:
            node.receive_money(send_money)

def get_final_dirty(transactions, dirty_addresses: list[str], intermediate_addresses: list[str], target_address: str):
    dirty_nodes = {address: Node(address, True) for address in dirty_addresses}
    intermediate_nodes = {address: Node(address, False) for address in intermediate_addresses}
    target_node = Node(target_address, False)

    from tqdm import tqdm
    for i in tqdm(range(len(transactions))):
        src = transactions.loc[i, 'from']
        dst = transactions.loc[i, 'to']
        value = transactions.loc[i, 'value']
        if src in dirty_addresses:
            if dst in intermediate_addresses:
                dirty_nodes[src].send_clear_money(value, intermediate_nodes[dst])
            elif dst == target_address:
                dirty_nodes[src].send_clear_money(value, target_node)
            elif dst in dirty_addresses:
                dirty_nodes[src].send_clear_money(value, dirty_nodes[dst])
            else:
                dirty_nodes[src].send_dirty_money(value)
        elif src in intermediate_addresses:
            if dst in intermediate_addresses:
                intermediate_nodes[src].send_clear_money(value, intermediate_nodes[dst])
            elif dst == target_address:
                intermediate_nodes[src].send_clear_money(value, target_node)
            elif dst in dirty_addresses:
                intermediate_nodes[src].send_clear_money(value, dirty_nodes[dst])
            else:
                intermediate_nodes[src].send_dirty_money(value)
        else:
            if dst in dirty_addresses:
                dirty_nodes[dst].receive_money({'dirty': value, 'clear': 0})
            elif dst in intermediate_addresses:
                intermediate_nodes[dst].receive_money({'dirty': 0, 'clear': value})
            elif dst in target_address:
                tartget_node.receive_money({'dirty': 0, 'clear': value})
            
    return dirty_nodes, intermediate_nodes, target_node

dirty_nodes, intermediate_nodes, tartget_node = get_final_dirty(df,['TXLWDuAP3F7xVjqfZPwGrnvTBYHTGYT1Qu', 'TPGZHxmFwHwNQHRHr5pb7SsdHJB4jVhi1k','TNj3LgXWLMp1oVdfCNKFkPKfUrCCqftcGr'], ['TWrxonF66TALQwJzPiJCqLstdZb3v533DY', 'TLRhiCwfqHbEdLqEDBsUA83JQPJAhyr7yQ','TL7QUBTHo9tmqdzyDXod3ikQuLhH75QFAS'], 'TTn2G7RQHmA3D6sjBHXGuS5NH4Jx7Te5Ni')


  5%|▍         | 13153/277051 [00:00<00:03, 66030.33it/s]

{'dirty': 0, 'clear': 10.0} {'dirty': 0.0, 'clear': 0.0}
{'dirty': 0, 'clear': 17067.0} {'dirty': 0.0, 'clear': 10.0}
{'dirty': 0, 'clear': 15105.74} {'dirty': 0.0, 'clear': 17077.0}
{'dirty': 0, 'clear': 37821.0} {'dirty': 0.0, 'clear': 2062.2599999999984}
{'dirty': 0, 'clear': 34355.0} {'dirty': 0.0, 'clear': 39883.259999999995}
{'dirty': 0, 'clear': 12431.0} {'dirty': 0.0, 'clear': 6364.959999999992}
{'dirty': 0, 'clear': 2325.0} {'dirty': 0.0, 'clear': 696.419999999991}
{'dirty': 0, 'clear': 10000.0} {'dirty': 0.0, 'clear': 3021.419999999991}
{'dirty': 0, 'clear': 3923.0} {'dirty': 0.0, 'clear': 9921.419999999991}
{'dirty': 0, 'clear': 15083.0} {'dirty': 0.0, 'clear': 13844.419999999991}
{'dirty': 0, 'clear': 7541.0} {'dirty': 0.0, 'clear': 28927.41999999999}
{'dirty': 0, 'clear': 7553.0} {'dirty': 0.0, 'clear': 6393.239999999991}
{'dirty': 0, 'clear': 7553.0} {'dirty': 0.0, 'clear': 13946.23999999999}
{'dirty': 0, 'clear': 19913.0} {'dirty': 0.0, 'clear': 21499.23999999999}
{'dirt

 10%|▉         | 26679/277051 [00:00<00:03, 66791.61it/s]

{'dirty': 0, 'clear': 41666.0} {'dirty': 109657005.76550502, 'clear': -109471154.61350499}
{'dirty': 0, 'clear': 127.89} {'dirty': 109636086.50550503, 'clear': -109429488.61350499}
{'dirty': 5154606.374460001, 'clear': -5011134.354460001} {'dirty': 109581769.96550502, 'clear': -109429360.72350499}
{'dirty': 143472.02, 'clear': 0.0} {'dirty': 114736376.33996502, 'clear': -114440495.07796499}
{'dirty': 0, 'clear': 25665.0} {'dirty': 114740879.899965, 'clear': -114440495.07796499}
{'dirty': 0, 'clear': 9798.5405} {'dirty': 114725954.529965, 'clear': -114414830.07796499}
{'dirty': 0, 'clear': 42540.0} {'dirty': 114714668.819965, 'clear': -114405031.53746499}
{'dirty': 0, 'clear': 37165.0} {'dirty': 114681770.75996502, 'clear': -114362491.53746499}
{'dirty': 5185540.3344600005, 'clear': -5142374.874460001} {'dirty': 114681770.75996502, 'clear': -114325326.53746499}
{'dirty': 43165.46, 'clear': 0.0} {'dirty': 119867311.09442502, 'clear': -119467701.41192499}
{'dirty': 0, 'clear': 9768.0} {'d

 15%|█▍        | 40239/277051 [00:00<00:03, 67133.51it/s]

...

 41%|████      | 113774/277051 [00:01<00:02, 66401.46it/s]

{'dirty': 0, 'clear': 17162.0} {'dirty': 5181538673.431517, 'clear': -5165414428.181469}
{'dirty': 0, 'clear': 0.0} {'dirty': 5181538673.431517, 'clear': -5165397266.181469}
{'dirty': 0, 'clear': 0.0} {'dirty': 5181533994.291516, 'clear': -5165397266.181469}
{'dirty': 0, 'clear': 0.0} {'dirty': 5181533994.291516, 'clear': -5165397266.181469}
{'dirty': 0, 'clear': 3000.0} {'dirty': 5181533994.291516, 'clear': -5165397266.181469}
{'dirty': 0, 'clear': 31323.0} {'dirty': 5181533994.291516, 'clear': -5165394266.181469}
{'dirty': 0, 'clear': 10000.0} {'dirty': 5181533994.291516, 'clear': -5165362943.181469}
{'dirty': 0, 'clear': 16393.0} {'dirty': 5181533994.291516, 'clear': -5165352943.181469}
{'dirty': 0, 'clear': 0.0} {'dirty': 5181457983.611517, 'clear': -5165336550.181469}
{'dirty': 0, 'clear': 0.0} {'dirty': 5181452583.611517, 'clear': -5165336550.181469}
{'dirty': 0, 'clear': 20038.0} {'dirty': 5181452583.611517, 'clear': -5165336550.181469}
{'dirty': 0, 'clear': 2973.0} {'dirty': 51

 46%|████▌     | 127114/277051 [00:01<00:02, 66367.34it/s]

{'dirty': 0, 'clear': 2928.25} {'dirty': 9066782189.43209, 'clear': -9050238895.919548}
{'dirty': 0, 'clear': 0.011747} {'dirty': 9066770441.58209, 'clear': -9050235967.669548}
{'dirty': 0, 'clear': 0.001174} {'dirty': 9066770441.58209, 'clear': -9050235967.6578}
{'dirty': 0, 'clear': 0.001174} {'dirty': 9066770441.58209, 'clear': -9050235967.656626}
{'dirty': 0, 'clear': 0.011747} {'dirty': 9066770441.58209, 'clear': -9050235967.65545}
{'dirty': 0, 'clear': 2000.0} {'dirty': 9066770441.58209, 'clear': -9050235967.643703}
{'dirty': 0, 'clear': 0.014326} {'dirty': 9066756114.942091, 'clear': -9050233967.643703}
{'dirty': 0, 'clear': 0.015759} {'dirty': 9066740355.632092, 'clear': -9050233967.629377}
{'dirty': 0, 'clear': 0.018624} {'dirty': 9066721730.992092, 'clear': -9050233967.613619}
{'dirty': 0, 'clear': 0.022922} {'dirty': 9066698808.362093, 'clear': -9050233967.594995}
{'dirty': 0, 'clear': 0.001432} {'dirty': 9066698808.362093, 'clear': -9050233967.572073}
{'dirty': 0, 'clear': 

100%|██████████| 277051/277051 [00:04<00:00, 66837.39it/s]


In [153]:
dirty_nodes['TPGZHxmFwHwNQHRHr5pb7SsdHJB4jVhi1k'].balance

Counter({'clear': 0.0, 'dirty': -126197337.84285545})

In [84]:
dirty_nodes['TXLWDuAP3F7xVjqfZPwGrnvTBYHTGYT1Qu'].send_clear_money(1231223)

In [85]:
dirty_nodes['TXLWDuAP3F7xVjqfZPwGrnvTBYHTGYT1Qu'].balance

Counter({'dirty': 0, 'clear': 0.0})